## csv 파일들을 합쳐서 각도 최소값 얻어서 비교

In [28]:
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np
import glob
import time
from collections import deque

import matplotlib.pyplot as plt

def calculate_angle(a,b,c):
    # 각 값을 받아 넘파이 배열로 변환
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    # 세 좌표를 가지고 각도로 변환하는 코드
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    # 180도가 넘으면 360에서 뺀 값을 계산한다.
    if angle > 180.0:
        angle = 360-angle
        
    return angle


# CSV 파일 병합 및 평균값 계산
file_pattern = "squat_csv_data/squat_*.csv"  # 100개 CSV 파일 경로
files = glob.glob(file_pattern)

# CSV 파일 병합
all_data = pd.concat([pd.read_csv(file) for file in files], ignore_index=True)

# 필요한 관절 ID
LEFT_SHOULDER, LEFT_HIP, LEFT_KNEE, LEFT_ANKLE = 11, 23, 25, 27
RIGHT_SHOULDER, RIGHT_HIP, RIGHT_KNEE, RIGHT_ANKLE = 12, 24, 26, 28

# 프레임별 각도 계산
knee_angles = []
torso_angles = []

for frame_id in all_data['frame'].unique():
    frame_data = all_data[all_data['frame'] == frame_id]

    # 좌표 추출
    left_shoulder = frame_data[frame_data['node_id'] == LEFT_SHOULDER][['x', 'y', 'z']].values.flatten()
    left_hip = frame_data[frame_data['node_id'] == LEFT_HIP][['x', 'y', 'z']].values.flatten()
    left_knee = frame_data[frame_data['node_id'] == LEFT_KNEE][['x', 'y', 'z']].values.flatten()
    left_ankle = frame_data[frame_data['node_id'] == LEFT_ANKLE][['x', 'y', 'z']].values.flatten()

    # 좌표 추출
    right_shoulder = frame_data[frame_data['node_id'] == RIGHT_SHOULDER][['x', 'y', 'z']].values.flatten()
    right_hip = frame_data[frame_data['node_id'] == RIGHT_HIP][['x', 'y', 'z']].values.flatten()
    right_knee = frame_data[frame_data['node_id'] == RIGHT_KNEE][['x', 'y', 'z']].values.flatten()
    right_ankle = frame_data[frame_data['node_id'] == RIGHT_ANKLE][['x', 'y', 'z']].values.flatten()

    # 좌표가 모두 있는 경우에만 계산
    if len(left_shoulder) and len(left_hip) and len(left_knee) and len(left_ankle):
        # 무릎 각도 (엉덩이-무릎-발목)
        knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
        if knee_angle < 170:
            knee_angles.append(knee_angle)

        # 상체 숙임 각도 (어깨-엉덩이-무릎)
        torso_angle = calculate_angle(left_shoulder, left_hip, left_knee)
        if torso_angle < 170:
            torso_angles.append(torso_angle)

# 평균 각도 계산
knee_mean_angle = np.min(knee_angles) if knee_angles else 90
torso_mean_angle = np.min(torso_angles) if torso_angles else 50

# 기준 각도 설정
KNEE_MIN, KNEE_MAX = knee_mean_angle, knee_mean_angle + 15
TORSO_MIN, TORSO_MAX = torso_mean_angle, torso_mean_angle + 20

print(f"무릎 기준 각도: {int(KNEE_MIN)}° ~ {int(KNEE_MAX)}°")
print(f"상체 숙임 기준 각도: {int(TORSO_MIN)}° ~ {int(TORSO_MAX)}°")

# Mediapipe 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils


# 웹캠 열기
cap = cv2.VideoCapture(1)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    feedback_summary = []  # 간단한 요약 피드백
    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        
        def get_point(landmark):
            return [landmarks[landmark].x, landmarks[landmark].y, landmarks[landmark].z]
        
        # 왼쪽 관절 좌표
        left_shoulder = get_point(mp_pose.PoseLandmark.LEFT_SHOULDER)
        left_hip = get_point(mp_pose.PoseLandmark.LEFT_HIP)
        left_knee = get_point(mp_pose.PoseLandmark.LEFT_KNEE)
        left_ankle = get_point(mp_pose.PoseLandmark.LEFT_ANKLE)
        
        # 각도 계산
        left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
        left_torso_angle = calculate_angle(left_shoulder, left_hip, left_knee)

        
        # 무릎 각도 피드백
        if KNEE_MIN <= left_knee_angle <= KNEE_MAX:
            feedback_summary.append("Knee angle: Correct")
        else:
            feedback_summary.append("Knee angle: Wrong")

        # 상체 숙임 각도 피드백
        if TORSO_MIN <= left_torso_angle <= TORSO_MAX:
            feedback_summary.append("Torso angle: Correct")
        else:
            feedback_summary.append("Torso angle: Wrong")


        # 피드백 출력
        y_offset = 20
        for i, line in enumerate(feedback_summary):
            color = (0, 255, 0) if "Correct" in line else (0, 0, 255)
            cv2.putText(image, line, (10, y_offset + i * 20),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

        # 각도 정보 출력
        cv2.putText(image, f"Knee Angle: {int(left_knee_angle)}°", (10, y_offset + 100),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
        cv2.putText(image, f"Torso Angle: {int(left_torso_angle)}°", (10, y_offset + 120),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

        # Mediapipe 관절 시각화
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    cv2.imshow("Squat Analysis", image)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



무릎 기준 각도: 60° ~ 75°
상체 숙임 기준 각도: 55° ~ 75°
